# Create Folders Structures
We download all sample dataset and the neccessary libraries from github

In [ ]:
import os
input_data = './samples/'
model_path = './models/'
prediction_path = './predicts/'
log_path = './logs/'

# Create the folder if it does not exist
os.makedirs(input_data, exist_ok=True)
os.makedirs(model_path, exist_ok=True)
os.makedirs(prediction_path, exist_ok=True)
os.makedirs(log_path, exist_ok=True)

# Load Libraries

In [ ]:
%pip install segmentation-models &> /dev/null
%load_ext tensorboard
!wget https://raw.githubusercontent.com/N-Jaro/segmentation_model_tutorial/main/unet_util.py

# https://stackoverflow.com/questions/75433717/module-keras-utils-generic-utils-has-no-attribute-get-custom-objects-when-im
# open the file keras.py, change all the 'init_keras_custom_objects' to 'init_tfkeras_custom_objects'.
# the location of the keras.py is in the error message. In your case, it should be in /usr/local/lib/python3.8/dist-packages/efficientnet/
!wget https://raw.githubusercontent.com/N-Jaro/segmentation_model_tutorial/main/keras.py
!cp './keras.py' '/usr/local/lib/python3.10/dist-packages/efficientnet/keras.py'

import shutil
import numpy as np
import tensorflow as tf
from keras import backend as K
import segmentation_models as sm
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from unet_util import dice_coef_loss, dice_coef, jacard_coef, dice_coef_loss, Residual_CNN_block, multiplication, attention_up_and_concatenate, multiplication2, attention_up_and_concatenate2, UNET_224, evaluate_prediction_result

sm.set_framework('tf.keras')
sm.framework()

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
--2023-08-01 20:18:20--  https://raw.githubusercontent.com/N-Jaro/segmentation_model_tutorial/main/unet_util.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12771 (12K) [text/plain]
Saving to: ‘unet_util.py.3’

unet_util.py.3      100%[===================>]  12.47K  --.-KB/s    in 0s      

2023-08-01 20:18:20 (29.0 MB/s) - ‘unet_util.py.3’ saved [12771/12771]

--2023-08-01 20:18:20--  https://raw.githubusercontent.com/N-Jaro/segmentation_model_tutorial/main/keras.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.1

'tf.keras'


# Start to create the model using Segmentation-model library



In [ ]:
# Avaiable backbones for Unet architechture
# 'vgg16' 'vgg19' 'resnet18' 'resnet34' 'resnet50' 'resnet101' 'resnet152' 'inceptionv3'
# 'inceptionresnetv2' 'densenet121' 'densenet169' 'densenet201' 'seresnet18' 'seresnet34'
# 'seresnet50' 'seresnet101' 'seresnet152', and 'attentionUnet'
backend = 'resnet50' # ResNet50 is the best model in the TL study

# Data location
# 'covington' 'rowancreek' 'alexander'
location = 'rowancreek'

# Added first Convo 8 to 3 channels layers to the random init model
name = 'ml-model-' + backend + '-' + str(np.random.randint(1000000))

logdir = log_path + name
if(os.path.isdir(logdir)):
  shutil.rmtree(logdir)
os.makedirs(logdir, exist_ok=True)

# Create the base model

This model is the Unet architechture with ResNet50 backbone. Since we do not use the ImageNet pretrain weights, we can create the model with 8 channels without adding an additional Convolution layer.

### Create Attention U-net model

This one is the model created by Zewei

In [ ]:
# Attention U-net model
learning_rate =0.0000359
model = UNET_224()

# Compile the model with 'Adam' optimizer and define the loss and metrics
model.compile(optimizer = Adam(learning_rate=learning_rate),
              loss = dice_coef_loss,
              metrics = [dice_coef,'accuracy'])

### Create the ImageNet backbone Unet model

In [ ]:
#initialize U-net model with 8 channels input
model = sm.Unet(backend, classes = 1, encoder_weights=None, input_shape=(None, None, 8))

# Compile the model with 'Adam' optimizer (0.001 is the default learning rate) and define the loss and metrics
model.compile(optimizer = Adam(),
              loss = dice_coef_loss,
              metrics=[dice_coef,'accuracy'])

NameError: ignored

In [ ]:
# prompt: get the weights of the model

weights = np.array(model.get_weights())
model.set_weights(weights + 1)

<ipython-input-15-93bc0b79c324>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  weights = np.array(model.get_weights())


### Prepare the hyper parameters

In [ ]:
# define hyperparameters and callback modules
patience = 10
maxepoch = 500
callbacks = [ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=patience, min_lr=1e-9, verbose=1, mode='min'),
             EarlyStopping(monitor='val_loss', patience=patience, verbose=0),
             ModelCheckpoint(model_path+name+'.h5', monitor='val_loss', save_best_only=True, verbose=0),
             TensorBoard(log_dir=logdir)]

# Load data

In [ ]:
X_train = np.load(input_data+location+'/train_data.npy').astype(np.float32)
Y_train = np.load(input_data+location+'/train_label.npy').astype(np.float32)
X_validation = np.load(input_data+location+'/vali_data.npy').astype(np.float32)
Y_validation = np.load(input_data+location+'/vali_label.npy').astype(np.float32)

# Train model

In [ ]:
train_history = model.fit(x = X_train,y = Y_train,
                          validation_data = (X_validation, Y_validation),
                          batch_size = 16, epochs = maxepoch, verbose=0, callbacks = callbacks)

# Use model to predict the test set

In [ ]:
# Load the best model saved by the callback module
from keras.models import load_model
best_model = load_model(model_path+name+'.h5',
                        custom_objects={'dice_coef_loss':dice_coef_loss,
                                        'dice_coef':dice_coef,})

# load the test data
X_test = np.load(input_data+location+'/bottom_half_test_data.npy').astype(np.float32)

# predict the test data using the loaded model
test_predicted= best_model.predict(X_test)

# convert the prediction probability to true or false with threshold at 0.5
test_predicted_threshold = (test_predicted > 0.5).astype(np.uint8)

# save the prediction results
np.save(prediction_path+name+'_predict.npy',test_predicted_threshold)
print('Predtion results saved: ' + prediction_path+name+'_predict.npy')

In [ ]:
tf.InteractiveSession()

AttributeError: ignored